In [2]:
import urllib, json
import urllib.request
import pandas as pd

In [43]:
def get_table(league_code, gw_start, gw_end):
    gw_start=gw_start-1
    currpage=1
    teams = {}
    while True:
        league_url = "https://fantasy.premierleague.com/api/leagues-classic/%d/standings/?page_new_entries=1&page_standings=%d&phase=1" % (league_code,currpage)
        response = urllib.request.urlopen(league_url)
        data = json.loads(response.read())
        for i in range(len(data.get('standings').get('results'))):
            team_id = data.get('standings').get('results')[i].get('entry')
            name = data.get('standings').get('results')[i].get('player_name')
            teams[team_id] = name
        currpage+=1
        if(data.get('standings').get('has_next'))==False:
            break
        
    rank=0
    prev=-1
    buffer=0
    df = pd.DataFrame(columns=['id', 'name', 'points'])
    for team_id in teams.keys():
        total = 0
        transfers = 0
        team_url = 'https://fantasy.premierleague.com/api/entry/%d/history/' % team_id
        response = urllib.request.urlopen(team_url)
        data = json.loads(response.read())
        for gw_no in range(gw_start, gw_end+1):
            try:
                total += data.get('current')[gw_no].get('points')
                total -= data.get('current')[gw_no].get('event_transfers_cost')
                transfers += data.get('current')[gw_no].get('event_transfers')
            except IndexError:
                total += 0
                transfers+=0
        if(total!=prev):
            prev=total
            rank+=buffer+1
            buffer=0
        else:
            buffer+=1
        d = pd.DataFrame({'id':team_id, 'name':teams[team_id], 'points':total, 'transfers':transfers}, index=[rank])
        
        df = df.append(d)
    df = df.sort_values(['points'], ascending=False)
    return df

In [44]:
get_table(807,1,2)

,id,name,points,transfers
1,1806,Dungeon Master,78,0.0
2,1353211,Sahil Thorve,76,0.0
3,3748368,Swapnil Markhedkar,73,0.0
4,1100199,Sahil Waykole,71,0.0
4,3343446,Rohan Deshpande,71,0.0
...,...,...,...,...
70,7548,Anshuman Deswal,21,0.0
73,1930070,Yatindra Jain,17,0.0
73,1466707,Aditya Datar,17,0.0
75,1835660,Rohit Kulkarni,12,0.0
